# Import Libraries

In [1]:
from swarmintelligence import MemoryBasedGreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
optimal_params

{'wolfSize': 25, 'maxIteration': 100, 'crossover_rate': 1.0}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    gwo_g = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    gwo_b = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.

1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627 

2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  29

3359.8652306428266  |  2004.6997362709596  |  2640.825119524543
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.825119524543
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.699736270959

1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553

5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[121, 178]","[94, 166]","[125, 182]",930.780721,2436.358480,1787.922710,...,"[[125, 255, 255, 255, 255, 255, 255, 255, 255,...","[930.7811479817464, 930.7811479817464, 930.781...","[2436.3584799702053, 2436.3584799702053, 2436....","[1787.9236630530743, 1787.9236630530743, 1787....","[1.783797264099121, 1.656202793121338, 1.95896...","[2293.7582194010415, 2293.7582194010415, 2293....","[47.893195961441556, 47.893195961441556, 47.89...","[14.525327230279872, 14.525327230279872, 14.52...","[0.7809698970724323, 0.7809698970724323, 0.780...","[0.941043257168062, 0.941043257168062, 0.94104..."
1,AirplaneF16.tiff,3,otsu,max,"[107, 158, 196]","[78, 138, 189]","[122, 173, 201]",971.128000,2534.074737,1856.439291,...,"[[122, 255, 201, 255, 255, 201, 201, 201, 201,...","[971.1290558159712, 971.1290558159712, 971.129...","[2534.075361427361, 2534.075361427361, 2534.07...","[1856.4464905049515, 1856.4198748283472, 1856....","[4.51273512840271, 3.3638978004455566, 3.48564...","[1572.9299570719402, 1572.7307955423992, 1573....","[39.6601810015025, 39.65767007203524, 39.67010...","[16.163709770459146, 16.164259701047374, 16.16...","[0.7344375813345811, 0.7343920008887942, 0.734...","[0.9584716610352234, 0.9585050798524942, 0.958..."
2,AirplaneF16.tiff,4,otsu,max,"[84, 127, 170, 200]","[67, 117, 165, 202]","[113, 151, 181, 204]",994.896619,2587.612959,1901.876773,...,"[[113, 204, 204, 255, 204, 204, 204, 204, 204,...","[994.9228621596616, 994.9167971396116, 994.916...","[2587.651267179317, 2587.6087197136694, 2587.6...","[1901.8096584477273, 1901.8973100660207, 1901....","[3.33612060546875, 3.305786609649658, 3.120724...","[1160.2566973368328, 1144.7434399922688, 1160....","[34.06254097005731, 33.83405739772085, 34.0686...","[17.485262768963988, 17.54372197406609, 17.483...","[0.7173998358423077, 0.7161500236492726, 0.717...","[0.9695165815943976, 0.9696727208879978, 0.969..."
3,AirplaneF16.tiff,5,otsu,max,"[76, 113, 149, 180, 202]","[60, 103, 142, 181, 206]","[109, 146, 171, 193, 208]",1006.660370,2618.815077,1923.362398,...,"[[109, 208, 208, 208, 208, 208, 208, 208, 208,...","[1006.933052245426, 1006.8833928369396, 1006.9...","[2619.0002508259377, 2619.0362799939194, 2619....","[1923.3295077310652, 1923.5110505386747, 1923....","[4.891283750534058, 3.9234070777893066, 3.7954...","[837.6814473470052, 857.9579467773438, 857.298...","[28.94272701987505, 29.290919186282697, 29.279...","[18.900014639095087, 18.796143596510642, 18.79...","[0.728726642343755, 0.7277618825788146, 0.7276...","[0.9772848592246767, 0.9770703731793143, 0.977..."
4,Lena.png,2,otsu,max,"[137, 196]","[78, 146]","[96, 141]",1004.913932,2393.647254,2181.865306,...,"[[141, 141, 141, 141, 141, 141, 141, 141, 141,...","[1004.9139322682681, 1004.9139322682681, 1004....","[2393.647254097627, 2393.647254097627, 2393.64...","[2181.8670821363294, 2181.8670821363294, 2181....","[4.767527341842651, 3.769305944442749, 3.66043...","[1858.9784520467122, 1858.9784520467122, 1858....","[43.115872391112674, 43.115872391112674, 43.11...","[15.43806005098941, 15.43806005098941, 15.4380...","[0.6394589731037388, 0.6394589731037388, 0.639...","[0.8954769944582632, 0.8954769944582632, 0.895..."
5,Lena.png,3,otsu,max,"[127, 176, 212]","[57, 107, 160]","[83, 110, 147]",1070.115598,2597.979378,2270.493498,...,"[[147, 147, 147, 147, 147, 147, 147, 147, 147,...","[1070.1183744124746, 1070.1183744124746, 1070....","[2597.9819871813006, 2597.9819871813006, 2597....","[2270.5129983240836, 2270.5129983240836, 2270....","[4.579118251800537, 5.0370025634765625, 3.7357...","[1171.812235514323, 1171.812235514323, 1167.56...","[34.231743097807964, 34.231743097807964, 34.16...","[17.442223324732417, 17.442223324732417, 17.45...","[0.699291

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_otsu_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_otsu_RGB_dict.pkl', 'wb'))